## Red Hat Product Documentation Verification

Using this notebook and the various dependencies, you can verify that the documentation was ingested properly inside the corresponding Milvus collection.

In [ ]:
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

In [ ]:
class product_info:
    def __init__(self, product, product_full_name, version, language):
        self.product = product
        self.product_full_name = product_full_name
        self.version = version
        self.language = language

In [ ]:
products = []
products.append(product_info('red_hat_openshift_ai_self-managed',
                             'Red Hat OpenShift AI Self-Managed',
                             '2.8',
                             'en-US'))
products.append(product_info('red_hat_openshift_ai_self-managed',
                             'Red Hat OpenShift AI Self-Managed',
                             '2.7',
                             'en-US'))
products.append(product_info('red_hat_openshift_ai_self-managed',
                             'Red Hat OpenShift AI Self-Managed',
                             '2.6',
                             'en-US'))
products.append(product_info('red_hat_3scale_api_management',
                             'Red Hat 3scale API Management',
                             '2.14',
                             'en-US'))
products.append(product_info('red_hat_advanced_cluster_management_for_kubernetes',
                             'Red Hat Advanced Cluster Management for Kubernetes',
                             '2.10',
                             'en-US'))
products.append(product_info('red_hat_advanced_cluster_security_for_kubernetes',
                             'Red Hat Advanced Cluster Security for Kubernetes',
                             '4.4',
                             'en-US'))
products.append(product_info('red_hat_amq_streams',
                             'Red Hat AMQ Streams',
                             '2.6',
                             'en-US'))
products.append(product_info('red_hat_ansible_automation_platform',
                             'Red Hat Ansible Automation Platform',
                             '2.4',
                             'en-US'))
products.append(product_info('red_hat_ansible_lightspeed_with_ibm_watsonx_code_assistant',
                             'Red Hat Ansible Lightspeed with IBM watsonx Code Assistant',
                             '2.x_latest',
                             'en-US'))
products.append(product_info('red_hat_data_grid',
                             'Red Hat Data Grid',
                             '8.4',
                             'en-US'))
products.append(product_info('red_hat_developer_hub',
                             'Red Hat Developer Hub',
                             '1.1',
                             'en-US'))
products.append(product_info('red_hat_enterprise_linux',
                             'Red Hat Enterprise Linux',
                             '9',
                             'en-US'))
products.append(product_info('red_hat_enterprise_linux',
                             'Red Hat Enterprise Linux',
                             '8',
                             'en-US'))
products.append(product_info('red_hat_build_of_microshift',
                             'Red Hat build of MicroShift',
                             '4.15',
                             'en-US'))
products.append(product_info('red_hat_openshift_data_foundation',
                             'Red Hat OpenShift Data Foundation',
                             '4.15',
                             'en-US'))
products.append(product_info('red_hat_satellite',
                             'Red Hat Satellite',
                             '6.14',
                             'en-US'))
products.append(product_info('red_hat_single_sign-on',
                             'Red Hat Single Sign-On',
                             '7.6',
                             'en-US'))
products.append(product_info('red_hat_advanced_cluster_security_for_kubernetes',
                             'Red Hat Advanced Cluster Security for Kubernetes',
                             '4.4',
                             'en-US'))
products.append(product_info('red_hat_enterprise_linux',
                             'Red Hat Enterprise Linux',
                             '7',
                             'en-US'))
products.append(product_info('openshift_container_platform',
                             'Red Hat OpenShift Container Platform',
                             '4.15',
                             'en-US'))
products.append(product_info('openshift_container_platform',
                             'Red Hat OpenShift Container Platform',
                             '4.14',
                             'en-US'))
products.append(product_info('openshift_container_platform',
                             'Red Hat OpenShift Container Platform',
                             '4.13',
                             'en-US'))
products.append(product_info('openshift_container_platform',
                             'Red Hat OpenShift Container Platform',
                             '4.12',
                             'en-US'))
products.append(product_info('red_hat_openshift_serverless',
                             'Red Hat OpenShift Serverless',
                             '1.32',
                             'en-US'))
products.append(product_info('red_hat_hybrid_cloud_console',
                             'Red Hat Hybrid Cloud Console',
                             '1-latest',
                             'en-US'))
products.append(product_info('red_hat_insights',
                             'Red Hat Insights',
                             '1-latest',
                             'en-US'))

In [ ]:
milvus = {}
milvus["MILVUS_HOST"] = "vectordb-milvus.milvus.svc.cluster.local"
milvus["MILVUS_PORT"] = 19530
milvus["MILVUS_USERNAME"] = os.getenv('MILVUS_USERNAME')
milvus["MILVUS_PASSWORD"] = os.getenv('MILVUS_PASSWORD')

In [ ]:
model_kwargs = {'trust_remote_code': True, 'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=True
)

In [ ]:
query = "How do I install this product?"

for product_info in products:
    MILVUS_COLLECTION = f"{product_info.product}_{product_info.language}_{product_info.version}".replace("-","_").replace(".","_")
    db = Milvus(
        embedding_function=embeddings,
        connection_args={"host": milvus["MILVUS_HOST"], "port": milvus["MILVUS_PORT"], "user": milvus["MILVUS_USERNAME"], "password": milvus["MILVUS_PASSWORD"]},
        collection_name=MILVUS_COLLECTION,
        metadata_field="metadata",
        text_field="page_content",
        auto_id=True,
        drop_old=False
        )
    docs_with_score = db.similarity_search_with_score(query)

    for doc, score in docs_with_score:
        print("-" * 80)
        print("Score: ", score)
        print(doc.page_content)
        print("-" * 80)